# Data Preperation Dependencies

In [1]:
import json
import os
import torch
import pandas as pd

# Data preperation

In [2]:
def create_datasets(default_directory: str):
    """
    Preprocess the data and labels to provide text pair

    Args:
        default_directory: Default directory for both training and validation data
    
    Returns:
        A dictionary contained processed training and validation sets
    """

    # Defining dictionary
    data_dict = {
        "train": [],
        "validation": []
    }

    # Iterate through folders
    for split in ["train", "validation"]:
        for difficulty in ["easy", "medium", "hard"]:
            # Difficulty dict
            difficulty_dict = os.path.join(default_directory, difficulty)
            # Set current directory [train, validation]
            current_directory = os.path.join(difficulty_dict, split)
            
            # Iterate over all filenames
            for filename in os.listdir(current_directory):
                # Only work on .txt files
                if filename.endswith(".txt"):
                    text_path = os.path.join(current_directory, filename)
                    label_path = os.path.join(current_directory, "truth-" + filename.replace(".txt", ".json"))

                    # Open an process the files
                    # Text files
                    with open(text_path) as f:
                        text = f.read()
                    paragraphs = text.strip().split("\n")
                    # Labels
                    with open(label_path) as f:
                        object = json.load(f)
                    labels = object.get("changes")
                    
                    # print(paragraphs)
                    # print(labels)

                    # Error handling by removing badly formatted files
                    if len(labels) != len(paragraphs)-1:
                        os.remove(text_path)
                        os.remove(label_path)
                        print("Removed bad formatted files")
                    
                    # Fill up data_dict
                    for i in range(1, len(paragraphs)):
                        data_dict[split].append([paragraphs[i-1], paragraphs[i], labels[i-1]])
    return data_dict


In [3]:
default_directory = "../pan24-multi-author-analysis"
data_dict = create_datasets(default_directory=default_directory)

In [51]:
df = pd.DataFrame(data_dict.get("train"), columns=["paragraph1", "paragraph2", "label"])
df.head()

,paragraph1,paragraph2,label
0,Just like who knew the Taliban would be just a...,Also the EU wasn’t born out of a “let’s preven...,1
1,Also the EU wasn’t born out of a “let’s preven...,Their money? What money did they have before t...,1
2,Their money? What money did they have before t...,Well...yeah actually. It kinda sucks to listen...,1
3,"In general, be courteous to others. Debate/dis...",Sigh. There are so many right wing monsters we...,1
4,Sigh. There are so many right wing monsters we...,r/politics is currently accepting new moderato...,1


# Custom Datasets

In [ ]:
# # Dependencies
# from torch.utils.data import Dataset, DataLoader

# # Creating custom dataset
# class CustomParagraphsDataset(Dataset):
#     """ 
#     Dataset containing pairs of data points 

#     Args:
#         Dataset

#     Return:
#         Sample dictionary
#     """

#     # Initialize the variables
#     def __init__(self, train=True, transform=None):
#         columns = ["paragraph1", "paragraph2", "label"]
#         self.dataframe = pd.DataFrame(data_dict.get("train"), columns=columns) if train else pd.DataFrame(data_dict.get("validation"), columns=columns)
#         self.transform = transform

#     # Function for returning size of dataset
#     def __len__(self):
#         return len(self.dataframe)

#     # Function for retrieving item
#     def __getitem__(self, idx):
#         # Convert index to list
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
        
#         # Defining sample
#         sample = {
#             "paragraph1": self.dataframe.iloc[idx]["paragraph1"],
#             "paragraph2": self.dataframe.iloc[idx]["paragraph2"],
#             "label": self.dataframe.iloc[idx]["label"]
#         }

#         # Transform check
#         if self.transform:
#             sample = self.transform(sample)
        
#         return sample

# TEST START ------

In [ ]:
# from datasets.dataset_dict import DatasetDict
# from datasets import Dataset

# columns = ["paragraph1", "paragraph2", "label"]

# d = {
#     "train": Dataset.from_dict({
#         "paragraph1": pd.DataFrame(data_dict.get("train"), columns=columns)["paragraph1"],
#         "paragraph2": pd.DataFrame(data_dict.get("train"), columns=columns)["paragraph2"],
#         "label": pd.DataFrame(data_dict.get("train"), columns=columns)["label"]
#     })
# }

# raw = DatasetDict(d)

In [ ]:
# train_dataset = CustomParagraphsDataset()
# validation_dataset = CustomParagraphsDataset(train=False)

In [6]:
# Importing dataset creation dependenceis
from datasets import DatasetDict, Dataset

# Defining column names
columns = ["paragraph1", "paragraph2", "label"]

# Creating raw dataset
raw_datasets = DatasetDict({
    "train": Dataset.from_dict({
        "paragraph1": pd.DataFrame(data_dict.get("train"), columns=columns)["paragraph1"],
        "paragraph2": pd.DataFrame(data_dict.get("train"), columns=columns)["paragraph2"],
        "label": pd.DataFrame(data_dict.get("train"), columns=columns)["label"]
    }),
    "validation": Dataset.from_dict({
        "paragraph1": pd.DataFrame(data_dict.get("validation"), columns=columns)["paragraph1"],
        "paragraph2": pd.DataFrame(data_dict.get("validation"), columns=columns)["paragraph2"],
        "label": pd.DataFrame(data_dict.get("validation"), columns=columns)["label"]
    })
})

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['paragraph1', 'paragraph2', 'label'],
        num_rows: 51962
    })
    validation: Dataset({
        features: ['paragraph1', 'paragraph2', 'label'],
        num_rows: 11194
    })
})

# Tokenizing and Encoder

In [36]:
from transformers import AutoTokenizer
reference = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(reference)

In [41]:
def tokenize_function(sample):
    return tokenizer(
        sample["paragraph1"],
        sample["paragraph2"],
        truncation=True
    )

In [42]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map: 100%|██████████| 11194/11194 [00:01<00:00, 10937.39 examples/s]


In [43]:
# Testing mapping
print(tokenized_datasets.column_names)

{'train': ['paragraph1', 'paragraph2', 'label', 'input_ids', 'attention_mask'], 'validation': ['paragraph1', 'paragraph2', 'label', 'input_ids', 'attention_mask']}


In [45]:
"""
ONLY FOR WHEN NOT USING THE TRAINER API
"""
# Post process removal
# for key in tokenized_datasets.keys():
#     tokenized_datasets[key] = tokenized_datasets[key].remove_columns(["paragraph1", "paragraph2"])
#     tokenized_datasets[key] = tokenized_datasets[key].rename_column("label", "labels")
#     tokenized_datasets[key] = tokenized_datasets[key].with_format("torch")

'\nONLY FOR WHEN NOT USING THE TRAINER API\n'

In [46]:
tokenized_datasets["train"]

Dataset({
    features: ['paragraph1', 'paragraph2', 'label', 'input_ids', 'attention_mask'],
    num_rows: 51962
})

# Trainer

In [47]:
def setup_model(MODEL_NAME: str, num_labels: int):
    from transformers import AutoModelForSequenceClassification
    model = AutoModelForSequenceClassification.from_pretrained(
        pretrained_model_name_or_path=MODEL_NAME,
        num_labels=num_labels
    )
    return model

In [57]:
MODEL_NAME = "roberta-base"
num_labels = len(pd.unique(df["label"]))
model = setup_model(MODEL_NAME=MODEL_NAME, num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Arguments for training
from transformers import TrainingArguments

args = TrainingArguments(
    "trainer", 
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32
)

# Model preperation

In [ ]:
# import torch
# import torch.nn as nn
# from transformers import RobertaForSequenceClassification, RobertaTokenizer

In [ ]:
# def create_model(MODEL_NAME: str):
#     """
#     Creates a RoBERTa model for the writing-style classification

#     Args:
#         model_name: Name of pre-trained model
#     Returns:
#         Model and tokenizer
#     """
#     model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
#     tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
#     return model, tokenizer

In [ ]:
# MODEL_NAME = "roberta-base"
# model, tokenizer = create_model(MODEL_NAME=MODEL_NAME)

In [ ]:
# train_encodings = tokenizer(
#     data_dict.get("train")[0][0],
#     truncuation=True,
#     padding=True
# )
# train_encodings

# Saving hyperparameters

In [ ]:
# from transformers import TrainingArguments
# training_args = TrainingArguments(
#     output_dir="roberta_trainer",
#     per_device_train_batch_size=32,
#     per_device_eval_batch_size=32,
#     num_train_epochs=5,
#     learning_rate=1e-3
# )

In [ ]:
# train_model()